Load NetCDF data

In [40]:
from netCDF4 import Dataset
import datetime
import glob
import os
import numpy as np
import json

DATA_HOME = "./data/"
def NetCDFtoJSON(variable="temp_mean", 
    start_date="20140101", end_date="20140131"):

    """ This function receives a time frame and produces a json version of
    the data stored at the netcdf repository.  """

    data_home = DATA_HOME

    start_date = datetime.datetime.strptime(start_date, "%Y%m%d").date()

    end_date = datetime.datetime.strptime(end_date, "%Y%m%d").date()

    data_files = glob.glob("%s/*.cdf" %(data_home))
    filtered_data = []
    for file_name in data_files:
        file_name = os.path.abspath(file_name)
        file_date = datetime.datetime.strptime(
            file_name.split('.')[2],"%Y%m%d").date()
        if file_date >= start_date and file_date <= end_date:
            file_obj = Dataset(file_name)
            date_value = {}
            variable_value_daily_mean = np.mean(file_obj.variables[variable][:])
            variable_value_daily_std = np.std(file_obj.variables[variable][:])
            date_value["date"] = file_date.isoformat()
            date_value["value"] = np.asscalar(variable_value_daily_mean)
            date_value["l"] = date_value["value"] - np.asscalar(variable_value_daily_std)
            date_value["u"] = date_value["value"] + np.asscalar(variable_value_daily_std)
            filtered_data.append(date_value)

    return json.dumps(filtered_data)

import json
from IPython.display import Javascript
#runs arbitrary javascript, client-side

start_time = "20141015"
end_time = "20151115"

Javascript("""
           window.chart_data_temp={0};
           window.chart_data_rh={1};
           window.chart_data_precip={2};
           """.format(NetCDFtoJSON("temp_mean", start_time, end_time),
                      NetCDFtoJSON("rh_mean", start_time, end_time),
                      NetCDFtoJSON("org_precip_rate_mean", start_time, end_time)))

<IPython.core.display.Javascript object>

Load JS libraries (and CSS)

In [41]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      mg: '//www.ncc.unesp.br/~winckler/metrics-graphics-2.7.0/dist/metricsgraphics.min'
  }
});

$('head').append('<link rel="stylesheet" type="text/css" href="//www.ncc.unesp.br/~winckler/metrics-graphics-2.7.0/dist/metricsgraphics.css">');

<IPython.core.display.Javascript object>

In [42]:
%%javascript
require(['d3', 'mg'], function(d3){

  $(".graph").remove();
  element.append("<div id='chart_temp' class='graph'></div>");
  element.append("<div id='chart_rh' class='graph'></div>");
  element.append("<div id='chart_precip' class='graph'></div>");
  $(".graph").width("600px");
  $(".graph").height("300px");

  window.mg_data_temp = MG.convert.date(chart_data_temp, 'date');
  window.mg_data_rh = MG.convert.date(chart_data_rh, 'date');
  window.mg_data_precip = MG.convert.date(chart_data_precip, 'date');

  MG.data_graphic({
        animate_on_load: true,
        title: "Line Chart",
        description: "This is a simple line chart.",
        data: window.mg_data_temp,
        width: 600,
        height: 300,
        right: 40,
        target: document.getElementById('chart_temp'),
        x_accessor: 'date',
        y_accessor: 'value',
        min_y_from_data: true,
        show_secondary_x_label: false,
        show_confidence_band: ['l', 'u'],
        area: false,
        linked: true,
    });
    
      MG.data_graphic({
        animate_on_load: true,
        title: "Line Chart",
        description: "This is a simple line chart.",
        data: window.mg_data_rh,
        width: 600,
        height: 300,
        right: 40,
        target: document.getElementById('chart_rh'),
        x_accessor: 'date',
        y_accessor: 'value',
        min_y_from_data: true,
        show_secondary_x_label: false,
        show_confidence_band: ['l', 'u'],
        area: false,
        linked: true,
    });
    
      MG.data_graphic({
        animate_on_load: true,
        title: "Line Chart",
        description: "This is a simple line chart.",
        data: window.mg_data_precip,
        width: 600,
        height: 300,
        right: 40,
        target: document.getElementById('chart_precip'),
        x_accessor: 'date',
        y_accessor: 'value',
        min_y_from_data: true,
        show_secondary_x_label: false,
        show_confidence_band: ['l', 'u'],
        area: false,
        linked: true,
    });
    
})


<IPython.core.display.Javascript object>